In [1]:
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import pandas as pd
import urllib.error
import urllib.request
import time
from config import REVIEW_START_ID, REVIEW_END_ID, URL_TEMPLATE, REQUEST_TIME_DELAY, HEADER_UA, HEADER_A, PATH_TO_SAVE_CSV

In [2]:
def get_request(url: str) -> str:
    req = urllib.request.Request(url)
    req.add_header('User-Agent', HEADER_UA)
    req.add_header('Accept', HEADER_A)
    req.add_header('Accept-Language', 'en-US,en;q=0.5')
    r = urllib.request.urlopen(req).read().decode('utf-8')
    time.sleep(REQUEST_TIME_DELAY)
    
    return r

def fill_data():
    data = []
    for ind in tqdm(range(REVIEW_START_ID, REVIEW_END_ID, 1)):
        try:
            response = get_request(URL_TEMPLATE + str(ind))
            vote = -1
            review = ""
            soup = bs(response, "html.parser")
            table = soup.find('table', {"class": "fullreview"})
            strong = table.find_all('strong')
            for text in strong:
                if not text.text.find("Vote:"):
                    vote = (text.text[6:])
            review = table.find_all('td')[-3].text
            if (vote == -1):
                continue
            if review == "":
                continue
            data.append([ind, vote, review])
        except:
            continue
    return data

In [3]:
data = fill_data()
columns=['Page ind', 'Rating', 'Text']
df = pd.DataFrame(data, columns=columns)
df.to_csv(PATH_TO_SAVE_CSV, index=False)

100%|██████████| 8122/8122 [4:30:37<00:00,  2.00s/it]  
